In [3]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from glob import glob

In [4]:
data_dir = "chest_xray"

train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

len(glob(f"{train_dir}/*/*")), len(glob(f"{test_dir}/*/*"))

(5216, 640)

In [9]:
import torchvision

torchvision_image = torchvision.io.read_image(glob(f"{train_dir}/*/*")[0])
print(type(torchvision_image), torchvision_image.shape)

<class 'torch.Tensor'> torch.Size([1, 1233, 1596])
